# doubledot.Salesforce
> Salesforce class for transfering data from Vantix to Salesforce

In [ ]:
#| default_exp crema_sf

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| exporti 
import io
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd
import os
from doubledot.ATMS_api import ATMS_api as ATMS
import time

## Salesforce @property sf_access_token @staticmethod list_files

In [ ]:
#| export
## Module for Salesforce API

class Salesforce:
    """Class for Salesforce API"""
    class_download_dir = os.path.join(os.getcwd(),'sf_download')
    class_upload_dir = os.path.join(os.getcwd(),'sf_upload')
    transfer_lock = False # lock to prevent multiple transfers at once - not implemented yet. but probably necessary to work with withh nbdev_test 

    ## Salesforce table relationships 
    ## these are also orderd by dependency
    model_d = {     'Contact'               :{ 'lookups_d': dict(), 'external_id':'contactId__c'},
                    'Membership__c'         :{ 'lookups_d': dict(), 'external_id':'membershipId__c'},
                    'MembershipTerm__c'     :{ 'lookups_d': {'membershipKey__c': 'Membership__c'}, 'external_id':'membershipTermId__c'},
                    'MembershipMember__c'   :{ 'lookups_d': {'contactKey__c': 'Contact', 'membershipTermKey__c': 'MembershipTerm__c' }, 'external_id':'membershipMemberId__c'},
                    # in SF change saleId__c to saleId__c
                    'Sale__c'               :{ 'lookups_d': {'booking_contactKey__c': 'Contact'}, 'external_id':'saleId__c'},
                    # in SF change membershipTermKey__c to membershipTermKey__c
                    'SaleDetail__c'         :{ 'lookups_d': {'membershipTermKey__c':'MembershipTerm__c', 'saleKey__c': 'Sale__c'}, 'external_id':'saleDetailId__c'},
                    # 'SaleDetail__c'         :{ 'lookups_d': {'membershipTermKey__c':'MembershipTerm__c', 'saleId__c': 'Sale__c'}, 'external_id':'saleDetailId__c'},
                    # in SF change tickeKey__c to ticketId__c
                    # change saleId__c to saleKey__c in Ticket__c
                    'Ticket__c'             :{ 'lookups_d': {'saleKey__c': 'Sale__c', 'saleDetailKey__c': 'SaleDetail__c'}, 'external_id':'ticketId__c'} 
            }

    def __init__(self):
        # set up access token 
        self._sf_access_token = self.get_token_with_REST()
        self.bulk_job_id = None
        self._atms = None

        # create unique download directory per instance
        if not os.path.exists(Salesforce.class_download_dir):
            os.makedirs(Salesforce.class_download_dir)
            print(f"Directory 'atms_download' created successfully.")
        else:
            print(f"Directory 'atms_download' already exists.")


    @property
    def sf_access_token(
        self 
     ) -> str : #the access toke
        """a @property
        retrieve token for Salesforce - verifies that token is still valid and attempts to get a new one from Salesforce site if not
        """
        if not(self.test_token()):
            self._sf_access_token = self.get_token_with_REST()
            # check to see if getting token worked
            assert (self.sf_access_token), "Fetching new token didn't fix problem"
        return self._sf_access_token


    @property
    def atms( self):
        if not self._atms:
            self._atms = ATMS()
        return self._atms
    
    @staticmethod
    def list_files():
        return os.listdir(Salesforce.class_download_dir)

show_doc(Salesforce.sf_access_token)


   

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/crema_sf.py#L61){target="_blank" style="float:right; font-size:smaller"}

### Salesforce.sf_access_token

>      Salesforce.sf_access_token ()

a @property
retrieve token for Salesforce - verifies that token is still valid and attempts to get a new one from Salesforce site if not

## get_token_with_REST

In [ ]:
#| export
@patch
def get_token_with_REST(self: Salesforce):
    """retieve the access token from Salesforce

    Returns:
        string: the access token 
    """
    with open('secrets.json') as f:
        secrets = json.load(f)
    
    DOMAIN = secrets['instance']
    payload = {
        'grant_type': 'password',
        'client_id': secrets['client_id'],
        'client_secret': secrets['client_secret'],
        'username': secrets['username'],
        'password': secrets['password'] + secrets['security_token']
    }
    oauth_url = f'{DOMAIN}/services/oauth2/token'

    auth_response = requests.post(oauth_url, data=payload)
    return auth_response.json().get('access_token') ######## <<<<<<<<<<<<<<<< .       



## test_token

In [ ]:
#| export
@patch
def test_token(self: Salesforce):
    """Verify that token is still valid. If it isn't, it attempts to get a new one.

    Returns:
        boolean: true if token is valid, false otherwise
    """
    sf_headers = { 'Authorization': f"Bearer {self._sf_access_token}", 'Content-Type': 'application/json' }
    end_point ="https://cremaconsulting-dev-ed.develop.my.salesforce.com"
    service = "/services/data/v57.0/"
    r = requests.request("GET", end_point+service+f"limits", headers=sf_headers, data={})
    valid_token = r.status_code == 200
    if not(valid_token): print(r.status_code, type(r.status_code))
    return valid_token
    


## create_job

In [ ]:
#| export
@patch
def create_job(self: Salesforce, 
                sf_object: str ='Contact', # the Salesforce object were going to operate on. 
                operation: str ='insert', # the database operation to use. Can be "insert","upsert" or "delete"
                external_id: str = 'External_Id__c' # when using "upsert", this field is used to identify the record
                )-> requests.Response :
    """Get job_id from Salesforce Bulk API

    """
    # Args: 
    #     sf_object (str, optional): the Salesforce object were going to operate on. Defaults to 'Contact'.
    #     operation (str, optional): ∆. Defaults to 'insert'.
    #     external_id (str, optional): the external id field for upsert operations. Defaults to 'External_Id__c'.
    #     sf_object (str, optional): the Salesforce object were going to operate on. Defaults to 'Contact'.
    #     operation (str, optional): the operation that will be used against the object. Defaults to 'insert'.
    #     external_id (str, optional): the external id field for upsert operations. Defaults to 'External_Id__c'.
    #     contentType (str, optional): the content type of the file. Defaults to 'CSV', 'JSON' also accepted.
    # Returns: 
    #     response: a response object containg the job_id. For more information on the response object see https://www.w3schools.com/python/ref_requests_response.asp
    #     a response object see https://www.w3schools.com/python/ref_requests_response.asp
        
    # Salesforce API docs: https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/create_job.htm    
    url = "https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest"

    # https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/datafiles_prepare_csv.htm
    ## we can set columnDelimiter to `,^,|,;,<tab>, and the default <comma>
    # sets the object to Contact, the content type to CSV, and the operation to insert
    payload_d = {
        "object": sf_object,
        "contentType": "CSV",
        # set columnDelimiter to TAB instead of comma for ease of dealing with commas in address fields
        #https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/create_job.htm
        "columnDelimiter": "TAB", 
        "operation": operation
    }
    
    # as per https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/walkthrough_upsert.htm
    if operation=='upsert':
        payload_d['externalIdFieldName']=external_id
    payload = json.dumps(payload_d)
    
    headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    # print(response.text)
    try:
        self.bulk_job_id = response.json()['id']
    except TypeError:
        print("Bad response in Salesforce.create_job :\n", response.text)
        
    print(f"Created job {self.bulk_job_id} for {sf_object} with operation {operation}") 
    return response 


## upload_csv

In [ ]:
#| export
@patch
def upload_csv(self : Salesforce, 
                obj_s: str = "", # Salesforce object to upload 
                use_download_dir_b : bool = False, # 
                # num_rows: int = 100, # the number of rows to upload 
                ) -> requests.Response:
    """Using the job_id from the previous step, upload the csv file to the job

    Args:
        file (filepointer): file pointer to the csv filek
    """
    # if not(file):
    #     # throw error
    #     assert False, "File not found"


    assert obj_s in ['Contact', 'Membership__c', 'MembershipTerm__c', 'MembershipMember__c', 'Sale__c', 'Ticket__c', 'SaleDetail__c']

    print(f"Uploading job {self.bulk_job_id} of object {obj_s}")

    if use_download_dir_b:
        file_path_s = os.path.join(Salesforce.class_download_dir , f"{obj_s}.csv")
    else:
        file_path_s = os.path.join(Salesforce.class_upload_dir , f"{obj_s}.csv")

    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}/batches"

    # replace all occurrences of '\2019' with \'
    # we may have done this in ATMS already, but just in case
    try:
        for line in fileinput.input(files=file_path_s, inplace=True):
            line = line.replace('\u2019', "'")
            print(line, end='') # this prints to the file instead of stdout (!!)

        with open(file_path_s,'r') as payload:
            headers = {
                'Content-Type': 'text/csv',
                'Authorization': f'Bearer {self.sf_access_token}'
                }
            response = requests.request("PUT", url, headers=headers, data=payload)
    except FileNotFoundError:
        print("File not found error in Saleforce.upload_csv: ", file_path_s)
        return None
    
    return response
   

## close_job

In [ ]:
#| export 
@patch
def close_job(self: Salesforce):
    # close the job (from Postman)
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}"

    payload = json.dumps({
        "state": "UploadComplete"
    })
    headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("PATCH", url, headers=headers, data=payload)

    # print(response.text)
    return response.json()
     

## job_status

In [ ]:
#| export       
# get job status (from Postman)
@patch
def job_status(self: Salesforce):
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}"

    payload = {}
    headers = {
    'Authorization': f'Bearer {self.sf_access_token}'
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()



## successful_results

In [ ]:
#| export
@patch
def successful_results(self : Salesforce):
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}/successfulResults"

    payload = {}
    headers = {
        'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    return response


## failed_result

In [ ]:
#| export
@patch
def failed_results(self: Salesforce):
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}/failedResults"

    payload = {}
    headers = {
        'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    try:
        display(pd.DataFrame(response.json()))
    except:
        print(response.text)
    # 
    return response


## get_sf_object_ids

In [ ]:
#| export
@patch
def get_sf_object_ids(self: Salesforce, 
                      object: str = 'Contact' # REST endpoint for data object
                      ):
    """Get Safesforce IDs for a the specified object

    """
    print(f"Retrieving Object Ids for {object} from Salesforce")
    sf_headers = { 'Authorization': f"Bearer {self.sf_access_token}", 'Content-Type': 'application/json' }
    end_point ="https://cremaconsulting-dev-ed.develop.my.salesforce.com"
    service = "/services/data/v57.0/"
    r = requests.request("GET", end_point+service+f"query/?q=SELECT+Id+FROM+{object}", headers=sf_headers, data={})
    assert isinstance(r.json(), dict), f"response: {r.json()}, header: {sf_headers}"
    object_ids = [d.get('Id') for d in r.json()['records']]
    while r.json()['done'] == False:
        new_url = end_point+r.json()['nextRecordsUrl']
        print(new_url)
        r = requests.request("GET", new_url, headers=sf_headers, data={})
        print((r.json()))
        fresh_object_ids = [d.get('Id') for d in r.json()['records']]
        print(len(fresh_object_ids))   
        object_ids+=fresh_object_ids
        
    print('total number of objects = ',len(object_ids))
    return object_ids


## delete_sf_objects

In [ ]:
#| export
@patch
def delete_sf_objects(self: Salesforce, 
                      obj_l: str | list = []
                      ):
    """Delete Salesforce objects from the specified list or string. Empty list deletes all objects."""
    # assert False, "want to catch test calls to this function"
    if isinstance(obj_l, str):
        obj_l = [obj_l]
    if len(obj_l) == 0:
        obj_l = Salesforce.model_d.keys()
    for obj_s in obj_l:
        print(f"Deleting {obj_s} objects from Salesforce")
        object_ids = self.get_sf_object_ids(obj_s)
        file_path_s = os.path.join(Salesforce.class_download_dir , f"{obj_s}.csv")
        print(f"In Salesforce.delete_sf_objects: Deleting {len(object_ids)} {obj_s} objects using {file_path_s}")
        with open(file_path_s, 'w') as f:
            f.write('Id\n')
            for id in object_ids:
                f.write(id+'\n')
                
        # force execute_job to use the csv file we just created        
        self.execute_job(obj_s, 'delete', use_ATMS_data=False)
        


In [ ]:
#| export
def escape_quotes(text):
    # Escape single quotes
    # text = re.sub(r"\'", r"\\'", text)
    text = re.sub(r"\'", r"_", text)
    # Escape double quotes
    text = re.sub(r'\"', r'_', text)
    # text = re.sub(r',', r'*', text) ## shouldn't be necessary with tab delimiter
    # text = re.sub(r'\"', r'\\"', text)
    return text.strip()

## process_memberships

In [ ]:
#| export

 #### modify so parent fields use correct shit 
#### maybe use a spreadsheet to make life easier 
# Name,Mother_Of_Child__r.contactId
# CustomObject1,123456

mem_s = "[].{membershipId__c: membershipId, \
    memberSince__c: memberSince, \
    updateDate__c: updateDate}"

memTerm_s = "[].membershipTerms[].{membershipTermId__c: membershipTermId,\
membershipKey__r_1_membershipId__c:membershipKey,\
effectiveDate__c:effectiveDate,\
expiryDate__c:expiryDate,\
membershipType__c:membershipType,\
upgradeFromTermKey__c:upgradeFromTermKey,\
giftMembership__c:giftMembership,\
refunded__c:refunded,\
saleDetailKey__c:saleDetailKey,\
itemKey__c:itemKey}"

memMembers_s = "[].membershipTerms[].membershipMembers[].{membershipMemberId__c:membershipMemberId,\
membershipTermKey__r_1_membershipTermId__c:membershipTermKey,\
cardNumber__c:cardNumber,\
membershipNumber__c:membershipNumber,\
cardStatus__c:cardStatus,\
contactKey__r_1_contactId__c:contactKey,\
displayName__c:displayName}"

@patch
def process_memberships(self: Salesforce ):
    """Unpack memberships data from atms object and write to membership, membership_terms, and membership_members csv files.
    
    We could modify this function to only process one of Memmbership, MembershipTerm, or MembershipMember.
    """
    # print("Processing memberships data")
    # custom objects need '__c' suffix
    mem_d = { 'memberships': {'fname':'Membership__c.csv', 'jmespath': mem_s},
               'membership_terms': {'fname':'MembershipTerm__c.csv','jmespath': memTerm_s},
               'membership_members': {'fname': 'MembershipMember__c.csv', 'jmespath': memMembers_s}
                }
            

    if not ('memberships' in self.atms.obj_d):
        self.atms.load_data_file_to_dict('memberships')
        assert 'memberships' in self.atms.obj_d, f"memberships not in atms.obj_d {self.atms.obj_d.keys()}"
    
    atms_d = self.atms.obj_d['memberships']

    for key, v_pair in mem_d.items():
        file_path_s = os.path.join(Salesforce.class_download_dir, v_pair['fname'])
        dict_l = jp.search(v_pair['jmespath'], atms_d)
        # print(f"Salesforce: Writing {len(dict_l)} {key} objects to {file_path_s}")
        with open(file_path_s, 'w') as f:
            if len(dict_l) == 0:
                # print(f"Warning: no {key} objects found")
                continue
            # hack to create header with a dot in it, jmespath won't do it
            f.write('\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()]) + '\n') # header
            for d in dict_l:
                #changed this to not write None for empty values, eg "" for null and false (a default value)
                f.write('\t'.join([str(v) if v else "" for v in d.values()]) + '\n')
    

## process_sales

In [ ]:
#| export


@patch
def process_sales(self: Salesforce ):
    search_s = "[].{saleId__c : saleKey,\
            saleAmount__c : saleAmount,\
            paymentAmount__c : paymentAmount,\
            saleDate__c : saleDate,\
            active__c : active,\
            terminalKey__c : terminalKey,\
            booking_bookingId__c   : booking.bookingId,\
            booking_bookingContactKey__c : booking.bookingContactKey,\
            booking_contactKey__r_1_contactId__c : booking.contactKey,\
            booking_contactIndividualKey__c : booking.contactIndividualKey,\
            booking_contactOrganizationKey__c : booking.contactOrganizationKey,\
            booking_displayName__c : booking.displayName,\
            booking_firstName__c : booking.firstName,\
            booking_lastName__c : booking.lastName,\
            booking_email__c : booking.email,\
            booking_phone__c : booking.phone}"
            # eventDate__c : eventDate,\
            # booking_contactKey__c : booking_contactKey,\
    
    assert 'sales' in self.atms.obj_d, f"sales not in atms.obj_d {self.atms.obj_d.keys()}"
    dict_l = jp.search(search_s, self.atms.obj_d['sales'])

    file_path_s = os.path.join(Salesforce.class_download_dir, 'Sale__c.csv')

    # print(f"Salesforce: Writing {len(dict_l)} 'Sales' objects to {file_path_s}")
    columnDelimiter = '\t'
    with open(file_path_s, 'w') as f:
        # hack to create header with a dot in it, jmespath won't do it
        header = '\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()])
        f.write(header + '\n') # header
        for item in dict_l:
            # changed this from single space to empty string if null
            l = [escape_quotes(str(v)) if v else "" for v in item.values()]
            f.write(columnDelimiter.join(l)+'\n')


        

## process_tickets



In [ ]:
#| export


@patch
def process_tickets(self: Salesforce ):
    search_s = "[].tickets[].{ticketId__c : ticketKey,\
        saleKey__r_1_saleId__c : saleKey,\
        saleDetailKey__r_1_saleDetailId__c : saleDetailKey,\
        itemDescription__c : itemDescription,\
        ticketDisplay__c : ticketDisplay}"
        # scheduleDate__c : scheduleDate,\
        # scheduleEndDate__c : scheduleEndDate,\

    assert 'sales' in self.atms.obj_d, f"sales not in atms.obj_d {self.atms.obj_d.keys()}"
    dict_l = jp.search(search_s, self.atms.obj_d['sales'])

    file_path_s = os.path.join(Salesforce.class_download_dir, 'Ticket__c.csv')

    # print(f"Salesforce: Writing {len(dict_l)} 'Ticket' objects to {file_path_s}")
    columnDelimiter = '\t'
    with open(file_path_s, 'w') as f:
        # hack to create header with a dot in it, jmespath won't do it
        header = '\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()])
        f.write(header + '\n') # header
        for item in dict_l:
            # changed this from single space to empty string if null
            l = [escape_quotes(str(v)) if v else "" for v in item.values()]
            f.write(columnDelimiter.join(l)+'\n')


        

## process_saleDetails

In [ ]:
#| export


@patch
def process_saleDetails(self: Salesforce ):
    search_s = "[].saleDetails[].{\
        saleDetailId__c : saleDetailId,\
        itemKey__c : itemKey,\
        scheduleKey__c : scheduleKey,\
        rateKey__c : rateKey,\
        categoryKey__c : categoryKey,\
        itemCategory__c : itemCategory,\
        pricingPriceKey__c : pricingPriceKey,\
        itemPrice__c : itemPrice,\
        itemTotal__c : itemTotal,\
        couponTotal__c : couponTotal,\
        discountTotal__c : discountTotal,\
        total__c : total,\
        revenueDate__c : revenueDate,\
        refundReason__c : refundReason,\
        refundReasonKey__c : refundReasonKey,\
        systemPriceOverride__c : systemPriceOverride,\
        membershipTermKey__r_1_membershipTermId__c : membershipTermKey,\
        saleKey__r_1_saleId__c : saleId}" 
    
    assert 'sales' in self.atms.obj_d, f"sales not in atms.obj_d {self.atms.obj_d.keys()}"
    dict_l = jp.search(search_s, self.atms.obj_d['sales'])

    file_path_s = os.path.join(Salesforce.class_download_dir, 'SaleDetail__c.csv')

    # print(f"Salesforce: Writing {len(dict_l)} 'SaleDetail' objects to {file_path_s}")
    columnDelimiter = '\t'
    with open(file_path_s, 'w') as f:
        # hack to create header with a dot in it, jmespath won't do it
        header = '\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()])
        f.write(header + '\n') # header
        for item in dict_l:
            # changed this from single space to empty string if null
            l = [escape_quotes(str(v)) if v else "" for v in item.values()]
            f.write(columnDelimiter.join(l)+'\n')


        

## process_contacts

In [ ]:
#| export
search_s = "[].{LastName: lastName,\
    FirstName: firstName,\
    MailingPostalCode: addresses[0].postalZipCode,\
    MailingCity: addresses[0].city,\
    MailingStreet: addresses[0].line1, \
    MailingCountry: addresses[0].country, \
    Phone: phones[?phoneType == 'Business'].phoneNumber | [0],\
    Email: emails[0].address[0],\
    contactId__c: contactId}"

import re



@patch
def process_contacts(self: Salesforce ):
    """ unpack contacts data from atms object and write to contacts csv file."""
    # print("process_contacts")
    if not ('contacts' in self.atms.obj_d):
        self.atms.load_data_file_to_dict('contacts')
        assert 'contacts' in self.atms.obj_d, f"contacts not in atms.obj_d {self.atms.obj_d.keys()}"
    
    file_path_s = os.path.join(Salesforce.class_download_dir, 'Contact.csv')
    dict_l = jp.search(search_s, self.atms.obj_d['contacts'])

    # if contact record has no LastName then
    for r in dict_l:
        if r['LastName'] == None:
            r['LastName'] = 'Not Provided'

    # print(f"Salesforce: Writing {len(dict_l)} 'Contact' objects to {file_path_s}")
    columnDelimiter = '\t'
    with open(file_path_s, 'w') as f:
        header = columnDelimiter.join(dict_l[0].keys())
        f.write(header+'\n')
        for item in dict_l:
            l = [escape_quotes(str(v)) if v else " " for v in item.values()]
            f.write(columnDelimiter.join(l)+'\n')

## process_objects


In [ ]:
#| export
@patch
def process_objects(self: Salesforce,
                    sf_object_s: str = "",
                    use_ATMS_data: bool = True
                    ):

    valid_obj_l = ['Contact', 'Membership__c', 'MembershipTerm__c', 'MembershipMember__c', 'Sale__c', 'Ticket__c','SaleDetail__c']
    if sf_object_s not in valid_obj_l:
        print(f"sf_object_s must be one of {', '.join(valid_obj_l)}")
        return
        
    print("Salesforce.process_objects: sf_object_s :",sf_object_s)

    if sf_object_s == 'SaleDetail__c' and use_ATMS_data:
        # this creates a file Sales__c.csv in the class_download_dir
        self.process_saleDetails()

    if sf_object_s == 'Ticket__c' and use_ATMS_data:
        # this creates a file Sales__c.csv in the class_download_dir
        self.process_tickets()

    if sf_object_s == 'Sale__c' and use_ATMS_data:
        # this creates a file Sales__c.csv in the class_download_dir
        self.process_sales()
    
    if sf_object_s == 'Contact' and use_ATMS_data:
        # this creates a file Contact.csv in the class_download_dir
        self.process_contacts()
    
    if sf_object_s in ['Membership__c', 'MembershipMember__c', 'MembershipTerm__c'] and use_ATMS_data:
        # this creates files Membership__c.csv, MembershipMember__c.csv, MembershipTerm__c.csv in the class_download_dir
        self.process_memberships()

## execute_job

In [ ]:
#| export

@patch
def execute_job(self: Salesforce, 
        sf_object_s : str = None, # Salesforce API object name
        operation : str = None, # REST operation, e.g. insert, upsert, delete
        max_trys : int = 20, # max number of times to try to get job status
        external_id : str = 'External_Id__c', # name of the field that is the unique identifier to ATMS
        use_ATMS_data : bool = True, # if True, update SF data directly from ATMS data, otherwise use previous data
        # **kwargs : dict # additional parameters to pass to the REST API
        ):
    """Test loading a Salesforce object with data from a local file"""
    print("execute_job")
    

    ## translate dictionaries to csv files
    self.process_objects(sf_object_s=sf_object_s, use_ATMS_data=use_ATMS_data)

    ## start data transfer to Salesforce server
    self.create_job(sf_object=sf_object_s, operation=operation, external_id=external_id)
    self.upload_csv(sf_object_s, use_download_dir_b = operation == 'delete')
    self.close_job()

    counter = 0
    sleep_time = 3
    
    job_status = self.job_status()['state']
    print("job status:", self.job_status()['state'])

    while job_status !='JobComplete' and job_status !='Failed' and counter < max_trys:
        print(f"waiting for job to complete, try {counter}, status: {self.job_status()['state']}")
        counter += 1
        time.sleep(sleep_time)
        job_status = self.job_status()['state']

    print("Failed results:")
    print(self.failed_results().text)

## get_fields from Salesforce for an object 

In [ ]:
#| export
@patch
def get_fields(self:Salesforce, 
               obj:str # the name of the Salesforce object
               ) -> requests.Response:
    """Get the fields for a given Salesforce object"""
    sf_headers = { 'Authorization': f"Bearer {self._sf_access_token}", 'Content-Type': 'application/json' }
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/sobjects/{obj}/describe"
    # print(url)
    response = requests.request("GET", url, headers=sf_headers)
    print(response)
    r = response.json()
    if response.status_code == 200:
        r = response.json()
        names = jp.search("fields[].name",r)
        return names
    else:
        raise Exception(f"Error: {response.status_code} {response.reason}")

## `retrieve_atms_records_by_contactId`

In [ ]:
#| export

@patch
def retrieve_atms_records_by_contactId(
    self: Salesforce, # the Salesforce object
    contactId_l : list # list of contactIds to retrieve
    ):
    """Retrieve ATMS records for a list of contactIds and write them to json files"""
    for obj in ['sales', 'contacts', 'memberships']:
        # does this not get written to file?, no. it does not. and we don't want it to because we're working on proccessing rn.
        self.atms.fetch_data_by_contactIds(obj, contactId_l) 

    # write data to json files
    self.atms.write_data_to_json_files()

## Test `retrieve_atms_records_by_contactId`

In [ ]:
#| eval: false
# 415645 73012 105765 and 342765 are also good contacts to bring in when you are rea and organization 133156
sf = Salesforce()
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167, 415645, 73012, 105765, 342765 ]
sf.retrieve_atms_records_by_contactId(pelton_ids)

# number of objects in dictionary
assert len(sf.atms.obj_d) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of objects in dictionary {sf.atms.obj_d.keys()}"
# number of files in directory
assert len(os.listdir(sf.atms.download_dir)) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of files in directory {sf.atms.download_dir}"



Directory 'atms_download' already exists.
Directory 'atms_download' already exists.
my id is n1pwa3hw
144164
we have contact_id 144164 and obj is sales
http://crm-api-telus.atmsplus.com/api/sales/contact/144164
[{'saleKey': '310928', 'saleAmount': '123.0500', 'paymentAmount': '123.0500', 'saleDate': '2005-08-19T10:23:33.03', 'active': True, 'terminalKey': 4, 'ticketCount': 0, 'eventDate': None, 'booking': {'bookingId': 48112, 'bookingContactKey': 192717, 'bookingContactType': 'Primary     ', 'contactKey': 144164, 'contactIndividualKey': 137810, 'contactOrganizationKey': 5649, 'displayName': 'Pelton, Donna', 'firstName': 'Donna', 'lastName': 'Pelton', 'email': 'tdpelton@shaw.ca', 'phone': '7804592956'}, 'saleComment': None, 'saleDetails': [{'saleDetailId': 760379, 'saleId': 310928, 'itemKey': 7, 'scheduleKey': None, 'rateKey': 3, 'categoryKey': None, 'itemQuantity': 1, 'pricingPriceKey': 207, 'itemPrice': 105.0, 'itemTotal': 105.0, 'couponTotal': 0.0, 'discountTotal': 0.0, 'total': 123.

In [ ]:
def remove_duplicates(file_path : str, idx: str = None):
    df = pd.read_csv('sf_download/'+file_path, sep='\t').drop_duplicates(subset=idx)
    df.to_csv('sf_download/'+file_path+'2',sep='\t', index=False)
    return df

def has_duplicates(file_s: str, idx: str = None):
    df = pd.read_csv('sf_download/'+file_s, sep='\t')
    return (df.shape)[0] != (df.drop_duplicates( subset=idx).shape)[0]

In [ ]:
#| export
## this will have only contacts that are members, and only members that are contacts. 
## is possible to have duplicates if a contact is a member of more than one membership?
## yes but the duplicated fields will only be in one group or the other
## so we separate them and then reduce them
## but we should really just make a function for this...

## given two dataframes and two fields return two dataframes that are subsets of the original dataframes for which the two fields match
def match_df(df1, df2, field1, field2):
    df1 = df1[df1[field1].isin(df2[field2])]
    df2 = df2[df2[field2].isin(df1[field1])]
    return df1, df2


In [ ]:
#| export
# test that all external ids are unique
# test that all lookups are valid

def test_lookup_fields(df_d):
    for fromKey in Salesforce.model_d.keys():
        # verify that external id is unique
        assert df_d[fromKey][Salesforce.model_d[fromKey]['external_id']].is_unique, f"external id not unique for {fromKey}"

        # verify that all lookups are valid 
        r_cols = [col for col in df_d[fromKey].columns if re.search('__r\.', col)]
        for col in r_cols:
            col_matches = re.search('(.*)__r\.(.*)', col)
            fromField = col
            lookupField = col_matches.group(1)+'__c'
            toField = col_matches.group(2)
            parentTable = Salesforce.model_d[fromKey]['lookups_d'][lookupField]
            fromColumn = df_d[fromKey][fromField]
            toColumn = df_d[parentTable][toField]
            assert (fromColumn.isin(toColumn)).all(), f"bad lookup: {fromKey} {fromField} {toField}"


## Perfect Data 

In [ ]:
#| exporti    
## these funcs all use the global variable df_d

# function that returns all the fields that point to a given foreign key
def fields_pointing_to_foreign_key(
        foreign_key : str, 
        df_d : dict
        ) -> list:
    return_list = []
    for name, df in df_d.items():
        for col in df.columns:
            if re.search(foreign_key[:-1], col) and  len(col)>len(foreign_key)+3:
                return_list.append((name, col))
    return return_list

# function that takes a foreign key and returns a set of all value that point to it
def get_pointing_foreign_key_values(
    foreign_key: str, 
    df_d : dict
    ) -> set:
    return_set = set()
    table_cols_l = fields_pointing_to_foreign_key(foreign_key, df_d)
    for table, col in table_cols_l:
        return_set.update(set(df_d[table][col]))
    return return_set


def reduce_to_referenced_rows(
    df_d : dict
    ) -> dict:
    """
    given a dictionary of dataframes, return a dictionary of dataframes where only rows whose foreign key is referenced by another table are kept
    """
    # df2_d should be a dictionary of perfect dataframes
    ## only rows whose foreign key is referenced by another table should be kept
    df2_d = {}
    for k, v in Salesforce.model_d.items():
        foreign_key = v['external_id']

        # keep all rows if no other table points to this one
        if len(fields_pointing_to_foreign_key(foreign_key, df_d)) == 0:
            df2_d[k] = df_d[k]
            print(k,'*', len(df_d[k]), len(df2_d[k]))
            continue
        point_to_foreign_key = get_pointing_foreign_key_values(foreign_key, df_d)
        # print(k, s) 
        keep_b = df_d[k][foreign_key].isin(point_to_foreign_key)
        df2_d[k] = df_d[k][keep_b]
        print(k,len(df_d[k]),  len(df2_d[k]))
    return df2_d

# i want to write these to file and send them to salesforce

## `perfect_data`

In [ ]:
#| export

# starting from atms object dictionary, create a dictionary of dataframes for all SF objects
# using this dictionary df_d, we can then remove duplicates of rows with same external_id
# and remove any row which has a lookup to a non-existent foreign key

@patch
def perfect_data(self: Salesforce) -> dict:
    assert len(self.atms.obj_d) == 4, 'atms dictionaries not available'
    obj_l = Salesforce.model_d.keys()
    for obj in obj_l:
        # write atms dictionaries to csv file, if dictionary there - otherwise exception
        self.process_objects(obj)

    # create a dictionary of dataframes for all SF objects  
    df_d = {}
    for i in Salesforce.model_d.keys(): 
        df_d[i] = pd.read_csv('sf_download/'+i+'.csv', sep='\t')
    try:
        print("this should fail")
        test_lookup_fields(df_d)    
    except:
        print("and it did. good.")
        
    for i in Salesforce.model_d.keys(): 
        # remove duplicates of rows with same external_id
        print("dropping duplicates for ", i, " on ", self.model_d[i]['external_id'],"...")
        df_d[i].drop_duplicates(subset= Salesforce.model_d[i]['external_id'], inplace=True)

    # remove any row which has a lookup to a non-existent foreign key
    for obj,relations in self.model_d.items():
        print(obj)
        for fromField, parent in relations['lookups_d'].items():
            parentExternalId = Salesforce.model_d[parent]['external_id']
            toColumn = df_d[parent][parentExternalId]

            # combine from field and parent external id to get Salesforce lookup field
            newFromField = fromField[:-1]+'r.'+parentExternalId
            fromColumn = df_d[obj][newFromField]
            indGood_b = fromColumn.isin(toColumn)
            good_b = indGood_b.sum() == len(indGood_b)
            if not good_b:
                # print('bad lookup: ', obj, newFromField, parentExternalId,len(indGood_b), len(indGood_b) - indGood_b.sum())
                df_d[obj]= match_df(df_d[obj], df_d[parent], newFromField, parentExternalId)[0]

    try:
        print("this should NOT fail")
        test_lookup_fields(df_d)
    except:
        print("but it did. Bad.")
        raise Exception("bad lookup")
    finally:
        print("finally, it did not fail. Good.")

    df2_d = reduce_to_referenced_rows(df_d)
    return df2_d

## `write_dict_to_csv`

In [ ]:
#| export 

@patch
def write_dict_to_csv(
    self: Salesforce,
    df2_d : dict
    ):
    # write dictionary of dataframes to upload directory for all SF objects  
    path = Salesforce.class_upload_dir
    for k in Salesforce.model_d.keys(): 
        with open(path+k+'.csv', 'w') as f:
            f.write(df2_d[k].to_csv(sep='\t', index=False))


## `upload_csv_to_sf`

In [ ]:
#| export

@patch
def upload_csv_to_sf(
    self : Salesforce,
    clean_sf : list | bool = [],
    clean_all : bool = False,
    operation : str = 'upsert'
    ):

    # clean SF if desired
    if clean_all:
        self.delete_sf_objects()
    else:
        for obj in clean_sf:
            self.delete_sf_objects(obj)

    # upload all data to SF
    for obj,relations in Salesforce.model_d.items():
        print(obj, relations['external_id'] )
        self.execute_job(obj, 'upsert', external_id=relations['external_id'], use_ATMS_data=False) 
        sleep(2)


# Testing Stuff

In [ ]:
display("""{mermaid}
graph TD;
    Membership-->MemberTerm;
    MemberTerm-->Member;
    Users-->Member;
    
""")

'{mermaid}\ngraph TD;\n    Membership-->MemberTerm;\n    MemberTerm-->Member;\n    Users-->Member;\n    \n'

```{mermaid}
graph TD;
    Membership-->MemberTerm;
    MemberTerm-->Member;
    Users-->Member;
    
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()